In [ ]:
import numpy as np
import pickle

In [ ]:
def load():
    with open("../../datasets/mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

(xtrain,ytrain,xtest,ytest)=load()
print(xtrain.shape,ytrain.shape)

In [24]:
class CNN:
    def __init__(self,alpha):
        self.hidden={}
        self.weights={}
        self.bias={}
        self.stride={}
        self.padding={}
        self.window={}
        self.count=0
        self.layers=[]
        self.layers_shape={}
        self.grad={}
        self.alpha=alpha
        

        
               
    def fit(self,x,y,epoch=1,batch_size=100):
        
        step=(int)(x.shape[0]/batch_size)
    
        for j in range(epoch):
            data_size=batch_size
            for i in range(step):
               
                
                batchx=x[(i)*batch_size:(i+1)*batch_size]
               
                batchy=y[(i)*batch_size:(i+1)*batch_size]
               
                yp=np.argmax(self.predict(batchx,batchy),axis=1)
                self.backpropagate(batchy)

                ya=np.argmax(batchy,axis=1)
                accuracy=(np.where(yp==ya)[0].shape[0]/yp.shape[0])
                
                #print("epoch=",j," step=",i," shape=",batchy.shape)
                
                print("epoch=",j," step=",i," accuracy=",accuracy)
                
                
          
            if(x.shape[0]%batch_size>0):
                batchx=x[step*batch_size:]
                batchy=y[step*batch_size:]
                
                yp=np.argmax(self.predict(batchx,batchy),axis=1)
                self.backpropagate(batchy) 

                ya=np.argmax(batchy,axis=1)
                accuracy=(np.where(yp==ya)[0].shape[0]/yp.shape[0])
                
                #print("epoch=",j," step=",i," shape=",batchy.shape)
                
                print("epoch=",j," step=",i," accuracy=",accuracy)
                
                
                  
        
        
    def predict(self,x,y):
        
        for i in range(len(self.layers)):
           
            if "input" in self.layers[i]:
                self.hidden[self.layers[i]]=x
            if "conv2d" in self.layers[i]:
                self.cnn_2d(i,False)
            elif "maxpool2d" in self.layers[i]:
                self.max_pool_2d(i,False)
            elif "avgpool2d" in self.layers[i]:
                self.avg_pool_2d(i,False)
            elif "dense" in self.layers[i]:
                self.dense_layer(i,False)
            elif "flatten" in self.layers[i]:
                self.flatten_layer(i,False)
            elif "relu" in self.layers[i]:
                self.relu_layer(i,False)
            elif "sigmoid" in self.layers[i]:
                self.sigmoid_layer(i,False)
            elif "softmax" in self.layers[i]:
                self.softmax_layer(i,False)
            elif "crossentropy" in self.layers[i]:
                self.cross_entropy_loss(i,False,y)  
            
            #print(self.layers[i],self.hidden[self.layers[i]])
      
        return self.hidden[self.layers[len(self.layers)-2]]
        
        
    
    def backpropagate(self,y):
            
       
        for i in range(len(self.layers)-1,0,-1):
          
           
            if "conv2d" in self.layers[i]:
                self.cnn_2d(i,True)
            elif "maxpool2d" in self.layers[i]:
                self.max_pool_2d(i,True)
            elif "avgpool2d" in self.layers[i]:
                self.avg_pool_2d(i,True)
            elif "dense" in self.layers[i]:
                self.dense_layer(i,True)
            elif "flatten" in self.layers[i]:
                self.flatten_layer(i,True)
            elif "relu" in self.layers[i]:
                self.relu_layer(i,True)
            elif "sigmoid" in self.layers[i]:
                self.sigmoid_layer(i,True)
            elif "softmax" in self.layers[i]:
                self.softmax_layer(i,True)
            elif "crossentropy" in self.layers[i]:
                self.cross_entropy_loss(i,True,y)

                
    #operations            
      
    def im2_col(self,x,window,stride):
        
        n1=(int)((x.shape[1]-window[0])/stride[0])+1
        n2=(int)((x.shape[2]-window[1])/stride[1])+1
       
        k1=window[0]
        k2=window[1]
        k3=window[2]
        
        (i)=np.fromfunction(lambda i:(i),(n1*n2*k1*k2*k3,),dtype=(int))


        tmp1=((i/(n2*k1*k2*k3)).astype(int))*stride[0]
        tmp2=((i%(n2*k1*k2*k3))/(k1*k2*k3)).astype(int)*stride[1]

        p=(i%(k1*k2*k3)/(k2*k3)).astype(int)

        p=p+tmp1
        q=tmp2+((i%(k2*k3))/k3).astype(int)

        r=i%k3
        
        x=x[:,p,q,r].reshape(x.shape[0],n1*n2,k1,k2,k3)
        return x
    
    
    
    def col2_img(self,grad,window,stride,shape):
        
        x=np.zeros(shape)
        
        n1=(int)((x.shape[1]-window[0])/stride[0])+1
        n2=(int)((x.shape[2]-window[1])/stride[1])+1
      
        k1=window[0]
        k2=window[1]
        k3=window[2]
        
        (i)=np.fromfunction(lambda i:(i),(n1*n2*k1*k2*k3,),dtype=(int))


        tmp1=((i/(n2*k1*k2*k3)).astype(int))*stride[0]
        tmp2=((i%(n2*k1*k2*k3))/(k1*k2*k3)).astype(int)*stride[1]

        p=(i%(k1*k2*k3)/(k2*k3)).astype(int)

        p=p+tmp1
        q=tmp2+((i%(k2*k3))/k3).astype(int)

        r=i%k3
        
        
       
        x=x.reshape(shape[0],shape[1]*shape[2]*shape[3])
        ind=(p+q*shape[1]+r*shape[2]*shape[3])

        
        for i in range(x.shape[0]):
            np.add.at(x[i],ind,grad[i])
            
        return x.reshape(shape[0],shape[1],shape[2],shape[3])
        

        
    def cnn_2d(self,layer,grad):
       
        
        x=self.hidden[self.layers[layer-1]]
      
        if(self.padding[self.layers[layer]]=='same'):
            padx=x.shape[1]*(self.stride[self.layers[layer]][0]-1)+self.weights[self.layers[layer]].shape[0]-self.stride[self.layers[layer]][0]
            pady=x.shape[2]*(self.stride[self.layers[layer]][1]-1)+self.weights[self.layers[layer]].shape[1]-self.stride[self.layers[layer]][1]
            
         
            if padx%2==0:
                padx1=padx/2
                padx2=padx/2
            else:
                padx1=(int)(padx/2)
                padx2=(int)(padx/2)+1
                
            if pady%2==0:
                pady1=pady/2
                pady2=pady/2
            else:
                pady1=(int)(pady/2)
                pady2=(int)(pady/2)+1
            
            x=np.pad(x, [(0,0),((int)(padx1), (int)(padx2)), ((int)(pady1), (int)(pady2)),(0,0)], mode='constant', constant_values=0)
            
            
        shapei=x.shape
        
       
        shapex=(int)((x.shape[1]-self.weights[self.layers[layer]].shape[0])/self.stride[self.layers[layer]][0])+1
        shapey=(int)((x.shape[2]-self.weights[self.layers[layer]].shape[1])/self.stride[self.layers[layer]][1])+1
        
        
        
       
        
        n=x.shape[0]
        d=self.weights[self.layers[layer]].shape[3]
        

        x=self.im2_col(x,self.weights[self.layers[layer]].shape,self.stride[self.layers[layer]])
        
        
        if grad==False:
            
            

            x=x.reshape(x.shape[0]*x.shape[1],x.shape[2]*x.shape[3]*x.shape[4])
            w=self.weights[self.layers[layer]].reshape(self.weights[self.layers[layer]].shape[0]*self.weights[self.layers[layer]].shape[1]*self.weights[self.layers[layer]].shape[2],self.weights[self.layers[layer]].shape[3])
            
          
            
            output=np.dot(x,w)+self.bias[self.layers[layer]]


            output=output.reshape(n,shapex,shapey,d)




            self.hidden[self.layers[layer]]=output
          

        else:
            
            w=self.weights[self.layers[layer]]
            shapew=w.shape
            w=w.reshape(w.shape[0]*w.shape[1]*w.shape[2],w.shape[3])
            
            shape=x.shape
            inp=x.reshape(x.shape[0]*x.shape[1],x.shape[2]*x.shape[3]*x.shape[4])
            tmp=self.grad[self.layers[layer+1]]
            grad=tmp.reshape(tmp.shape[0]*tmp.shape[1]*tmp.shape[2],tmp.shape[3])
            
            #print("n=",n)
            #print(grad)
            dw=np.dot(inp.T,grad)/n

            db=grad.sum(axis=0)/n
            
               
            #print("convw=",dw)
            #print("convb=",db)
            
            grad=np.dot(grad,w.T).reshape(shape[0],shape[1]*shape[2]*shape[3]*shape[4])
            

            
            self.weights[self.layers[layer]]-=self.alpha*dw.reshape(shapew[0],shapew[1],shapew[2],shapew[3])
            self.bias[self.layers[layer]]-=self.alpha*db
            
            tmp=self.col2_img(grad,(self.weights[self.layers[layer]].shape[0],self.weights[self.layers[layer]].shape[1],self.weights[self.layers[layer]].shape[2]),self.stride[self.layers[layer]],shapei)
            
            
            
            
            self.grad[self.layers[layer]]=tmp[:,padx1:tmp.shape[1]-padx2,pady1:tmp.shape[2]-pady2,:]
            
            
    def max_pool_2d(self,layer,grad):
        x=self.hidden[self.layers[layer-1]]
 
        shape=x.shape
        shapex=(int)((x.shape[1]-self.window[self.layers[layer]][0])/self.stride[self.layers[layer]][0])+1
        shapey=(int)((x.shape[2]-self.window[self.layers[layer]][1])/self.stride[self.layers[layer]][1])+1

        n=x.shape[0]
        d=x.shape[3]
        x=self.im2_col(x,(self.window[self.layers[layer]][0],self.window[self.layers[layer]][1],d),self.stride[self.layers[layer]])
        
        x=np.transpose(x,(0,1,4,2,3))
        x=x.reshape(x.shape[0],x.shape[1],x.shape[2],x.shape[3]*x.shape[4])
        
        if grad==False:

            output=np.max(x,axis=3)

            output=output.reshape(x.shape[0],shapex,shapey,d)

            self.hidden[self.layers[layer]]=output
      
        
        else:

            n_classes=shape[1]*shape[2]
            
            
            grad=self.grad[self.layers[layer+1]]

            a = np.arange(n_classes)
            b = np.zeros((n_classes, n_classes))
            b[np.arange(n_classes), a] = 1
            
            grad=b[np.argmax(x,axis=3)]*grad.reshape(grad.shape[0],grad.shape[1]*grad.shape[2],grad.shape[3],1)
            
            grad=np.transpose(grad,(0,1,3,2)).reshape(grad.shape[0]*grad.shape[1],grad.shape[2]*grad.shape[3])
            
            
            self.grad[self.layers[layer]]=self.col2_img(grad,(self.window[self.layers[layer]][0],self.window[self.layers[layer]][1],d),self.stride[self.layers[layer]],shape)
            
            
            
            
                        
                        
    def avg_pool_2d(self,layer,grad):
      
        x=self.hidden[self.layers[layer-1]]
        shape=x.shape
        shapex=(int)((x.shape[1]-self.window[self.layers[layer]][0])/self.stride[self.layers[layer]][0])+1
        shapey=(int)((x.shape[2]-self.window[self.layers[layer]][1])/self.stride[self.layers[layer]][1])+1

        n=x.shape[0]
        d=x.shape[3]
        
        x=self.im2_col(x,(self.window[self.layers[layer]][0],self.window[self.layers[layer]][1],d),self.stride[self.layers[layer]])
        
        x=np.transpose(x,(0,1,4,2,3))
        
       
        x=x.reshape(x.shape[0],x.shape[1],x.shape[2],x.shape[3]*x.shape[4])
        
        if grad==False:
            
            output=np.sum(x,axis=3)/(self.window[self.layers[layer]][0]*self.window[self.layers[layer]][1])

            output=output.reshape(x.shape[0],shapex,shapey,d)
            
            self.hidden[self.layers[layer]]=output
           
        
        else:
            #print(x.shape)
            
            tmp=(1/(np.sum(x,axis=3)+0.00001)).reshape(x.shape[0],x.shape[1],x.shape[2],1)
            #print(np.sum(x,axis=3))
            #print("avg2=",tmp)
            grad=self.grad[self.layers[layer+1]]
            
            #print("avg1=",grad)
            tmp2=grad.reshape(grad.shape[0],grad.shape[1]*grad.shape[2],grad.shape[3],1)

            grad=x*tmp*tmp2
                     
            grad=np.transpose(grad,(0,1,3,2)).reshape(grad.shape[0],grad.shape[1]*grad.shape[2]*grad.shape[3])
            
            #print("avg=",grad)
            self.grad[self.layers[layer]]=self.col2_img(grad,(self.window[self.layers[layer]][0],self.window[self.layers[layer]][1],d),self.stride[self.layers[layer]],shape)

        
        
                        
        
    def dense_layer(self,layer,grad):
        if grad==False:
            
            self.hidden[self.layers[layer]]=np.dot(self.hidden[self.layers[layer-1]],self.weights[self.layers[layer]])+self.bias[self.layers[layer]]
        else:
            
            dw=np.dot(self.hidden[self.layers[layer-1]].T,self.grad[self.layers[layer+1]])/self.hidden[self.layers[layer-1]].shape[0]

            db=self.grad[self.layers[layer+1]].sum(axis=0)/self.hidden[self.layers[layer]].shape[0]

            self.grad[self.layers[layer]]=np.dot(self.grad[self.layers[layer+1]],self.weights[self.layers[layer]].T)
            
            #print("dw=",dw)
            #print("db=",db)
            
            #print("dense=",self.grad[self.layers[layer]])
            self.weights[self.layers[layer]]-=self.alpha*dw
            self.bias[self.layers[layer]]-=self.alpha*db

    
    def flatten_layer(self,layer,grad):

        x=self.hidden[self.layers[layer-1]]
   
        if grad==False:
            self.hidden[self.layers[layer]]=x.reshape(x.shape[0],x.shape[1]*x.shape[2]*x.shape[3])
        else:
            
            self.grad[self.layers[layer]]=self.grad[self.layers[layer+1]].reshape(x.shape[0],x.shape[1],x.shape[2],x.shape[3])
    
        
    def relu_layer(self,layer,grad):
        if grad==False:
            self.hidden[self.layers[layer]]=self.hidden[self.layers[layer-1]].copy()
            self.hidden[self.layers[layer]][self.hidden[self.layers[layer]]<0]=0.
        else:
            self.grad[self.layers[layer]]=self.grad[self.layers[layer+1]].copy()
            self.grad[self.layers[layer]][self.hidden[self.layers[layer]]<0]=0.    
                
        
    def sigmoid_layer(self,layer,grad):
        if grad==False:
            #print("sigmoid_shape=",(1/(1+np.exp(-self.hidden[self.layers[layer-1]]))).shape)
            self.hidden[self.layers[layer]]=1/(1+np.exp(-self.hidden[self.layers[layer-1]]))
        else:
            self.grad[self.layers[layer]]=self.hidden[self.layers[layer]]*(1-self.hidden[self.layers[layer]])*self.grad[self.layers[layer+1]]
       
        
    def softmax_layer(self,layer,grad):
        if grad==False:
            shape=self.hidden[self.layers[layer-1]].shape[0]
            
            self.hidden[self.layers[layer]]=np.exp(self.hidden[self.layers[layer-1]])/np.sum(np.exp(self.hidden[self.layers[layer-1]]),axis=1).reshape(shape,1)
            #print(self.hidden[self.layers[layer]].sum(axis=1))
        else:
            self.grad[self.layers[layer]]=self.hidden[self.layers[layer]]*(1-self.hidden[self.layers[layer]])*self.grad[self.layers[layer+1]]
    
    
    
    def cross_entropy_loss(self,layer,grad,y):
        if grad==False:
            
            self.hidden[self.layers[layer]]=np.sum(-np.log(self.hidden[self.layers[layer-1]])*self.hidden[self.layers[layer-1]],axis=1)
           
        else:
            
            self.grad[self.layers[layer]]=-(1/self.hidden[self.layers[layer]][np.argmax(y,axis=1)]).reshape(y.shape[0],1)*y
           
            
                        
   

    #initialization

                        
    def input_layer(self,size):
        i="input_"+str(self.count)
        self.layers.append(i)
        self.layers_shape[i]=(size)
        self.count=self.count+1
    
    
    
    
    def conv2d(self,kernel,number,stride,padding):
        i="conv2d_"+str(self.count)
        self.layers.append(i)
        
         
        shape=(kernel[0],kernel[1],self.layers_shape[self.layers[self.count-1]][2],number)
        self.weights[i]=np.random.normal(loc=0.0, scale=1.0, size=shape[0]*shape[1]*shape[2]*shape[3]).reshape(shape[0],shape[1],shape[2],shape[3])
       
       
        self.bias[i]=np.zeros((1,number))
        self.stride[i]=stride
        self.padding[i]=padding

        if padding=="valid":
            shapex=(int)((self.layers_shape[self.layers[self.count-1]][0]-self.weights[i].shape[0])/self.stride[i][0])+1
            shapey=(int)((self.layers_shape[self.layers[self.count-1]][1]-self.weights[i].shape[1])/self.stride[i][1])+1
        else:
            shapex=self.layers_shape[self.layers[self.count-1]][0]
            shapey=self.layers_shape[self.layers[self.count-1]][1]
        self.layers_shape[i]=([shapex,shapey,number])
        
        self.count=self.count+1
        
        
                        
    def maxpool2d(self,window,stride):
        i="maxpool2d_"+str(self.count)
        self.layers.append(i)
        
        self.window[i]=window
        self.stride[i]=stride
    
        shapex=(int)((self.layers_shape[self.layers[self.count-1]][0]-self.window[i][0])/self.stride[i][0])+1
        shapey=(int)((self.layers_shape[self.layers[self.count-1]][1]-self.window[i][1]+1)/self.stride[i][1])+1
        
        self.layers_shape[i]=([shapex,shapey,self.layers_shape[self.layers[self.count-1]][2]])
        self.count=self.count+1
        
                        
                        
    def avgpool2d(self,window,stride):
        i="avgpool2d_"+str(self.count)
        self.layers.append(i)
        
        self.window[i]=window
        self.stride[i]=stride
    
        shapex=(int)((self.layers_shape[self.layers[self.count-1]][0]-self.window[i][0])/self.stride[i][0])+1
        shapey=(int)((self.layers_shape[self.layers[self.count-1]][1]-self.window[i][1]+1)/self.stride[i][1])+1
        
        self.layers_shape[i]=([shapex,shapey,self.layers_shape[self.layers[self.count-1]][2]])
        self.count=self.count+1
        
        
        
        
    def dense(self,size):
        i="dense_"+str(self.count)
        self.layers.append(i)
        
        self.weights[i]=np.random.normal(loc=0.0, scale=1.0, size=self.layers_shape[self.layers[self.count-1]]*size).reshape(self.layers_shape[self.layers[self.count-1]],size)
        
    
                    
        self.bias[i]=np.zeros((1,size))
        self.layers_shape[i]=(size)
        self.count=self.count+1
                        
    def flatten(self):
        
        i="flatten_"+str(self.count)
        self.layers.append(i)
        size=self.layers_shape[self.layers[self.count-1]][0]*self.layers_shape[self.layers[self.count-1]][1]*self.layers_shape[self.layers[self.count-1]][2]
        self.layers_shape[i]=(size)
        self.count=self.count+1
    
    def relu(self):
        i="relu_"+str(self.count)
        self.layers.append(i)
        self.layers_shape[i]=(self.layers_shape[self.layers[self.count-1]])
        self.count=self.count+1
    
    def sigmoid(self):
        i="sigmoid_"+str(self.count)
        self.layers.append(i)
        self.layers_shape[i]=(self.layers_shape[self.layers[self.count-1]])
        self.count=self.count+1
                        
    def softmax(self):
        i="softmax_"+str(self.count)
        self.layers.append(i)
        self.layers_shape[i]=(self.layers_shape[self.layers[self.count-1]])
        self.count=self.count+1
        
    def crossentropy(self):
        i="crossentropy_"+str(self.count)
        self.layers.append(i)
        self.layers_shape[i]=1
        self.count=self.count+1
        
        
        
    def summary(self):
    
        for i in self.layers:
            print("name:",i)
            print("layer_shape",":",self.layers_shape[i])
            try:
                print("stride:",self.stride[i])
                print("kernel:",self.weights[i].shape)
                print("padding:",self.padding[i])
            except Exception as e:
                print(e)
                try:
                    print("window:",self.window[i].shape)
                except:
                    b=2
            print("")
            
            

In [ ]:
n_classes=10
a = np.arange(n_classes)
b = np.zeros((n_classes, n_classes))
b[np.arange(n_classes), a] = 1


x=xtrain.reshape(xtrain.shape[0],28,28,1)
y=b[ytrain]

print(x.shape,y.shape)
classifier=CNN(0.01)

classifier.input_layer([x.shape[1],x.shape[2],x.shape[3]])
classifier.conv2d(kernel=(3,3),number=5,stride=(1,1),padding='same')
classifier.relu()
classifier.maxpool2d(window=(2,2),stride=(2,2))
classifier.conv2d(kernel=(3,3),number=10,stride=(2,2),padding='same')
classifier.relu()
classifier.maxpool2d(window=(2,2),stride=(2,2))
classifier.flatten()
classifier.dense(5)
classifier.sigmoid()

classifier.dense(10)
classifier.softmax()
classifier.crossentropy()
print(classifier.summary())

classifier.fit(x,y,batch_size=100,epoch=10)

#print(classifier.predict(x,y))

(60000, 28, 28, 1) (60000, 10)
name: input_0
layer_shape : [28, 28, 1]
'input_0'

name: conv2d_1
layer_shape : [28, 28, 5]
stride: (1, 1)
kernel: (3, 3, 1, 5)
padding: same

name: relu_2
layer_shape : [28, 28, 5]
'relu_2'

name: maxpool2d_3
layer_shape : [14, 14, 5]
stride: (2, 2)
'maxpool2d_3'

name: conv2d_4
layer_shape : [14, 14, 10]
stride: (2, 2)
kernel: (3, 3, 5, 10)
padding: same

name: relu_5
layer_shape : [14, 14, 10]
'relu_5'

name: maxpool2d_6
layer_shape : [7, 7, 10]
stride: (2, 2)
'maxpool2d_6'

name: flatten_7
layer_shape : 490
'flatten_7'

name: dense_8
layer_shape : 5
'dense_8'

name: sigmoid_9
layer_shape : 5
'sigmoid_9'

name: dense_10
layer_shape : 10
'dense_10'

name: softmax_11
layer_shape : 10
'softmax_11'

name: crossentropy_12
layer_shape : 1
'crossentropy_12'

None


C:\Users\asus\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:420: RuntimeWarning: overflow encountered in exp
C:\Users\asus\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:281: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


epoch= 0  step= 0  accuracy= 0.11
epoch= 0  step= 1  accuracy= 0.06
epoch= 0  step= 2  accuracy= 0.07
epoch= 0  step= 3  accuracy= 0.12
epoch= 0  step= 4  accuracy= 0.14
epoch= 0  step= 5  accuracy= 0.1
epoch= 0  step= 6  accuracy= 0.08
epoch= 0  step= 7  accuracy= 0.08
epoch= 0  step= 8  accuracy= 0.11
epoch= 0  step= 9  accuracy= 0.16
epoch= 0  step= 10  accuracy= 0.16
epoch= 0  step= 11  accuracy= 0.11
epoch= 0  step= 12  accuracy= 0.12
epoch= 0  step= 13  accuracy= 0.09
epoch= 0  step= 14  accuracy= 0.13
epoch= 0  step= 15  accuracy= 0.08
epoch= 0  step= 16  accuracy= 0.1
epoch= 0  step= 17  accuracy= 0.12
epoch= 0  step= 18  accuracy= 0.12
epoch= 0  step= 19  accuracy= 0.04
epoch= 0  step= 20  accuracy= 0.06
epoch= 0  step= 21  accuracy= 0.12
epoch= 0  step= 22  accuracy= 0.08
epoch= 0  step= 23  accuracy= 0.12
epoch= 0  step= 24  accuracy= 0.17
epoch= 0  step= 25  accuracy= 0.13
epoch= 0  step= 26  accuracy= 0.09
epoch= 0  step= 27  accuracy= 0.08
epoch= 0  step= 28  accuracy= 0.

epoch= 0  step= 232  accuracy= 0.11
epoch= 0  step= 233  accuracy= 0.09
epoch= 0  step= 234  accuracy= 0.09
epoch= 0  step= 235  accuracy= 0.08
epoch= 0  step= 236  accuracy= 0.04
epoch= 0  step= 237  accuracy= 0.11
epoch= 0  step= 238  accuracy= 0.08
epoch= 0  step= 239  accuracy= 0.09
epoch= 0  step= 240  accuracy= 0.08
epoch= 0  step= 241  accuracy= 0.11
epoch= 0  step= 242  accuracy= 0.1
epoch= 0  step= 243  accuracy= 0.13
epoch= 0  step= 244  accuracy= 0.13
epoch= 0  step= 245  accuracy= 0.12
epoch= 0  step= 246  accuracy= 0.15
epoch= 0  step= 247  accuracy= 0.08
epoch= 0  step= 248  accuracy= 0.15
epoch= 0  step= 249  accuracy= 0.1
epoch= 0  step= 250  accuracy= 0.1
epoch= 0  step= 251  accuracy= 0.15
epoch= 0  step= 252  accuracy= 0.1
epoch= 0  step= 253  accuracy= 0.2
epoch= 0  step= 254  accuracy= 0.11
epoch= 0  step= 255  accuracy= 0.11
epoch= 0  step= 256  accuracy= 0.08
epoch= 0  step= 257  accuracy= 0.15
epoch= 0  step= 258  accuracy= 0.19
epoch= 0  step= 259  accuracy= 0.

epoch= 0  step= 461  accuracy= 0.11
epoch= 0  step= 462  accuracy= 0.09
epoch= 0  step= 463  accuracy= 0.06
epoch= 0  step= 464  accuracy= 0.06
epoch= 0  step= 465  accuracy= 0.08
epoch= 0  step= 466  accuracy= 0.11
epoch= 0  step= 467  accuracy= 0.08
epoch= 0  step= 468  accuracy= 0.11
epoch= 0  step= 469  accuracy= 0.11
epoch= 0  step= 470  accuracy= 0.15
epoch= 0  step= 471  accuracy= 0.1
epoch= 0  step= 472  accuracy= 0.1
epoch= 0  step= 473  accuracy= 0.1
epoch= 0  step= 474  accuracy= 0.08
epoch= 0  step= 475  accuracy= 0.08
epoch= 0  step= 476  accuracy= 0.14
epoch= 0  step= 477  accuracy= 0.05
epoch= 0  step= 478  accuracy= 0.09
epoch= 0  step= 479  accuracy= 0.08
epoch= 0  step= 480  accuracy= 0.11
epoch= 0  step= 481  accuracy= 0.09
epoch= 0  step= 482  accuracy= 0.08
epoch= 0  step= 483  accuracy= 0.13
epoch= 0  step= 484  accuracy= 0.1
epoch= 0  step= 485  accuracy= 0.12
epoch= 0  step= 486  accuracy= 0.11
epoch= 0  step= 487  accuracy= 0.15
epoch= 0  step= 488  accuracy= 0

epoch= 1  step= 93  accuracy= 0.05
epoch= 1  step= 94  accuracy= 0.06
epoch= 1  step= 95  accuracy= 0.17
epoch= 1  step= 96  accuracy= 0.1
epoch= 1  step= 97  accuracy= 0.08
epoch= 1  step= 98  accuracy= 0.07
epoch= 1  step= 99  accuracy= 0.1
epoch= 1  step= 100  accuracy= 0.05
epoch= 1  step= 101  accuracy= 0.12
epoch= 1  step= 102  accuracy= 0.11
epoch= 1  step= 103  accuracy= 0.07
epoch= 1  step= 104  accuracy= 0.06
epoch= 1  step= 105  accuracy= 0.09
epoch= 1  step= 106  accuracy= 0.06
epoch= 1  step= 107  accuracy= 0.09
epoch= 1  step= 108  accuracy= 0.07
epoch= 1  step= 109  accuracy= 0.11
epoch= 1  step= 110  accuracy= 0.07
epoch= 1  step= 111  accuracy= 0.13
epoch= 1  step= 112  accuracy= 0.16
epoch= 1  step= 113  accuracy= 0.11
epoch= 1  step= 114  accuracy= 0.05
epoch= 1  step= 115  accuracy= 0.13
epoch= 1  step= 116  accuracy= 0.06
epoch= 1  step= 117  accuracy= 0.08
epoch= 1  step= 118  accuracy= 0.09
epoch= 1  step= 119  accuracy= 0.08
epoch= 1  step= 120  accuracy= 0.08
e

epoch= 1  step= 322  accuracy= 0.06
epoch= 1  step= 323  accuracy= 0.12
epoch= 1  step= 324  accuracy= 0.11
epoch= 1  step= 325  accuracy= 0.07
epoch= 1  step= 326  accuracy= 0.05
epoch= 1  step= 327  accuracy= 0.08
epoch= 1  step= 328  accuracy= 0.11
epoch= 1  step= 329  accuracy= 0.06
epoch= 1  step= 330  accuracy= 0.11
epoch= 1  step= 331  accuracy= 0.04
epoch= 1  step= 332  accuracy= 0.05
epoch= 1  step= 333  accuracy= 0.1
epoch= 1  step= 334  accuracy= 0.11
epoch= 1  step= 335  accuracy= 0.07
epoch= 1  step= 336  accuracy= 0.06
epoch= 1  step= 337  accuracy= 0.08
epoch= 1  step= 338  accuracy= 0.1
epoch= 1  step= 339  accuracy= 0.08
epoch= 1  step= 340  accuracy= 0.09
epoch= 1  step= 341  accuracy= 0.09
epoch= 1  step= 342  accuracy= 0.05
epoch= 1  step= 343  accuracy= 0.06
epoch= 1  step= 344  accuracy= 0.06
epoch= 1  step= 345  accuracy= 0.12
epoch= 1  step= 346  accuracy= 0.13
epoch= 1  step= 347  accuracy= 0.12
epoch= 1  step= 348  accuracy= 0.07
epoch= 1  step= 349  accuracy=

epoch= 1  step= 551  accuracy= 0.09
epoch= 1  step= 552  accuracy= 0.1
epoch= 1  step= 553  accuracy= 0.09
epoch= 1  step= 554  accuracy= 0.11
epoch= 1  step= 555  accuracy= 0.08
epoch= 1  step= 556  accuracy= 0.12
epoch= 1  step= 557  accuracy= 0.11
epoch= 1  step= 558  accuracy= 0.11
epoch= 1  step= 559  accuracy= 0.05
epoch= 1  step= 560  accuracy= 0.15
epoch= 1  step= 561  accuracy= 0.06
epoch= 1  step= 562  accuracy= 0.06
epoch= 1  step= 563  accuracy= 0.03
epoch= 1  step= 564  accuracy= 0.09
epoch= 1  step= 565  accuracy= 0.06
epoch= 1  step= 566  accuracy= 0.03
epoch= 1  step= 567  accuracy= 0.06
epoch= 1  step= 568  accuracy= 0.09
epoch= 1  step= 569  accuracy= 0.06
epoch= 1  step= 570  accuracy= 0.07
epoch= 1  step= 571  accuracy= 0.1
epoch= 1  step= 572  accuracy= 0.13
epoch= 1  step= 573  accuracy= 0.18
epoch= 1  step= 574  accuracy= 0.12
epoch= 1  step= 575  accuracy= 0.08
epoch= 1  step= 576  accuracy= 0.18
epoch= 1  step= 577  accuracy= 0.08
epoch= 1  step= 578  accuracy=

epoch= 2  step= 183  accuracy= 0.1
epoch= 2  step= 184  accuracy= 0.09
epoch= 2  step= 185  accuracy= 0.1
epoch= 2  step= 186  accuracy= 0.1
epoch= 2  step= 187  accuracy= 0.12
epoch= 2  step= 188  accuracy= 0.06
epoch= 2  step= 189  accuracy= 0.12
epoch= 2  step= 190  accuracy= 0.09
epoch= 2  step= 191  accuracy= 0.1
epoch= 2  step= 192  accuracy= 0.12
epoch= 2  step= 193  accuracy= 0.09
epoch= 2  step= 194  accuracy= 0.06
epoch= 2  step= 195  accuracy= 0.12
epoch= 2  step= 196  accuracy= 0.07
epoch= 2  step= 197  accuracy= 0.06
epoch= 2  step= 198  accuracy= 0.1
epoch= 2  step= 199  accuracy= 0.08
epoch= 2  step= 200  accuracy= 0.06
epoch= 2  step= 201  accuracy= 0.05
epoch= 2  step= 202  accuracy= 0.08
epoch= 2  step= 203  accuracy= 0.16
epoch= 2  step= 204  accuracy= 0.08
epoch= 2  step= 205  accuracy= 0.07
epoch= 2  step= 206  accuracy= 0.07
epoch= 2  step= 207  accuracy= 0.07
epoch= 2  step= 208  accuracy= 0.13
epoch= 2  step= 209  accuracy= 0.07
epoch= 2  step= 210  accuracy= 0.

epoch= 2  step= 412  accuracy= 0.12
epoch= 2  step= 413  accuracy= 0.11
epoch= 2  step= 414  accuracy= 0.13
epoch= 2  step= 415  accuracy= 0.16
epoch= 2  step= 416  accuracy= 0.11
epoch= 2  step= 417  accuracy= 0.12
epoch= 2  step= 418  accuracy= 0.12
epoch= 2  step= 419  accuracy= 0.11
epoch= 2  step= 420  accuracy= 0.12
epoch= 2  step= 421  accuracy= 0.11
epoch= 2  step= 422  accuracy= 0.12
epoch= 2  step= 423  accuracy= 0.13
epoch= 2  step= 424  accuracy= 0.17
epoch= 2  step= 425  accuracy= 0.11
epoch= 2  step= 426  accuracy= 0.14
epoch= 2  step= 427  accuracy= 0.13
epoch= 2  step= 428  accuracy= 0.14
epoch= 2  step= 429  accuracy= 0.13
epoch= 2  step= 430  accuracy= 0.2
epoch= 2  step= 431  accuracy= 0.12
epoch= 2  step= 432  accuracy= 0.12
epoch= 2  step= 433  accuracy= 0.17
epoch= 2  step= 434  accuracy= 0.1
epoch= 2  step= 435  accuracy= 0.17
epoch= 2  step= 436  accuracy= 0.1
epoch= 2  step= 437  accuracy= 0.15
epoch= 2  step= 438  accuracy= 0.1
epoch= 2  step= 439  accuracy= 0